<a href="https://colab.research.google.com/github/mr-ayush-agrawal/DL_Notebooks/blob/main/Lect/Hyperprams_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hyperparameters Tuning

We are using [Pima Indians Diabetes Dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data) for this notebook and tring to get the concept of hyperparameter tuning

A copy of same dataset has been stored in the github and using the same here.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mr-ayush-agrawal/DL_Notebooks/main/Lect/IndianDiabetes.csv')
df.shape

(768, 9)

In [3]:
df.sample(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
296,2,146,70,38,360,28.0,0.337,29,1
100,1,163,72,0,0,39.0,1.222,33,1
555,7,124,70,33,215,25.5,0.161,37,0
270,10,101,86,37,0,45.6,1.136,38,1
169,3,111,90,12,78,28.4,0.495,29,0


In [4]:
df.corr().Outcome

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x.shape, y.shape

((768, 8), (768,))

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = len(x))

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [9]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = x.shape[1]))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1377 (5.38 KB)
Trainable params: 1377 (5.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.fit(x_train, y_train, batch_size = 32, epochs = 100, validation_data = (x_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 13ms/step - loss: 0.6196 - accuracy: 0.6987 - val_loss: 0.6351 - val_accuracy: 0.6299
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5609 - accuracy: 0.7410 - val_loss: 0.5980 - val_accuracy: 0.6818
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5197 - accuracy: 0.7687 - val_loss: 0.5748 - val_accuracy: 0.6948
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4921 - accuracy: 0.7801 - val_loss: 0.5561 - val_accuracy: 0.7143
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4722 - accuracy: 0.7801 - val_loss: 0.5416 - val_accuracy: 0.7013
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4597 - accuracy: 0.7818 - val_loss: 0.5394 - val_accuracy: 0.7013
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4514 - accuracy: 0.7850 - val_loss: 0.5383 - val_accuracy: 0.7143
Epoch 8/100


# Tuning

using the library called **Keras Tuner** for the purpose of Hyperparam tuning

In [15]:
!pip install keras-tuner -U > /dev/null
!pip show keras-tuner

Name: keras-tuner
Version: 1.4.7
Summary: A Hyperparameter Tuning Library for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The KerasTuner authors
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: keras, kt-legacy, packaging, requests
Required-by: 


In [16]:
import kerastuner as kt

## Tuning Optimizers

In [17]:
# here just selecting the optimizers
def build_model (hp):
    model = Sequential()
    model.add(Dense(32, activation = 'relu', input_dim = 8))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

    optimizers = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer = optimizers, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [18]:
# making the tuner object
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

In [19]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 00m 15s


In [20]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [21]:
model = tuner.get_best_models(num_models = 1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1377 (5.38 KB)
Trainable params: 1377 (5.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 100,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 17ms/step - loss: 0.4788 - accuracy: 0.7769 - val_loss: 0.5563 - val_accuracy: 0.7078
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4644 - accuracy: 0.7834 - val_loss: 0.5469 - val_accuracy: 0.7208
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4545 - accuracy: 0.7915 - val_loss: 0.5372 - val_accuracy: 0.7338
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4472 - accuracy: 0.7932 - val_loss: 0.5309 - val_accuracy: 0.7532
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4421 - accuracy: 0.7915 - val_loss: 0.5243 - val_accuracy: 0.7468
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4392 - accuracy: 0.7932 - val_loss: 0.5212 - val_accuracy: 0.7468
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4354 - accuracy: 0.7915 - val_loss: 0.5208 - val_accuracy: 0.7338
Epoch 14

## Tuning the numeber of neurons

In [23]:
def build_model(hp):
    model = Sequential()
    units = hp.Int("Units", 8, 128)  #, step_size = 8

    model.add(Dense(units, activation='relu', input_dim=x.shape[1]))
    # model.add(Dense(units, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    return model

In [24]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 7,
                        directory = 'mydir')

In [25]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 7 Complete [00h 00m 02s]
val_accuracy: 0.649350643157959

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 13s


In [26]:
tuner.get_best_hyperparameters()[0].values

{'Units': 84}

In [27]:
model = tuner.get_best_models(num_models =1)[0]

In [28]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 100,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5497 - accuracy: 0.7720 - val_loss: 0.5671 - val_accuracy: 0.7273
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5190 - accuracy: 0.7671 - val_loss: 0.5516 - val_accuracy: 0.7273
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4976 - accuracy: 0.7769 - val_loss: 0.5426 - val_accuracy: 0.7273
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4820 - accuracy: 0.7769 - val_loss: 0.5369 - val_accuracy: 0.7143
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4703 - accuracy: 0.7866 - val_loss: 0.5302 - val_accuracy: 0.7273
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4621 - accuracy: 0.7866 - val_loss: 0.5267 - val_accuracy: 0.7273
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4558 - accuracy: 0.7850 - val_loss: 0.5240 - val_accuracy: 0.7273
Epoch 14

## Number of Layers

In [29]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(79, activation = 'relu', input_dim = x.shape[1]))
    for i in range(hp.Int('Layers', min_value = 1, max_value = 10)):
        model.add(Dense(79, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    return model

In [30]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 10,
                        directory = 'mydir',
                        project_name = 'Layers')

In [31]:
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 9 Complete [00h 00m 04s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 34s


In [32]:
tuner.get_best_hyperparameters()[0].values

{'Layers': 8}

In [33]:
model = tuner.get_best_models(num_models =1)[0]

In [34]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 100,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/100
20/20 [==============================] - 3s 25ms/step - loss: 0.3985 - accuracy: 0.8257 - val_loss: 0.5397 - val_accuracy: 0.7273
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3785 - accuracy: 0.8355 - val_loss: 0.6075 - val_accuracy: 0.7078
Epoch 9/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3834 - accuracy: 0.8241 - val_loss: 0.5859 - val_accuracy: 0.6948
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3603 - accuracy: 0.8420 - val_loss: 0.6056 - val_accuracy: 0.6883
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3541 - accuracy: 0.8355 - val_loss: 0.6338 - val_accuracy: 0.6818
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3256 - accuracy: 0.8648 - val_loss: 0.6475 - val_accuracy: 0.7143
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.2983 - accuracy: 0.8779 - val_loss: 0.7356 - val_accuracy: 0.6818
Epoch 1

## Multiple Parameters

Finding the optimal value of multiple parameters

In [35]:
def build_model (hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layer', min_value = 1, max_value = 10)) :
        if counter == 0 :
            model.add(
                      Dense(
                            units = hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 4),
                            activation = hp.Choice('activation'+str(i), values = ['relu', 'tanh', 'sigmoid', 'elu']),
                            input_dim = x.shape[1]
                            )
                      )
        else :
            model.add(
                      Dense(
                            units = hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 4),
                            activation = hp.Choice('activation'+str(i), values = ['relu', 'tanh', 'sigmoid', 'elu'])
                            )
                    )
        model.add(Dropout(hp.Choice('dropout'+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter += 1

    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = hp.Choice('Optimizer', values = ['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])


    return model


In [36]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials = 50,
                        directory = 'mydir',
                        project_name = 'final')

In [37]:
tuner.search(x_train, y_train, epochs = 15, validation_data = (x_test, y_test))

Trial 50 Complete [00h 00m 03s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 04m 09s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 6,
 'units0': 40,
 'activation0': 'elu',
 'dropout0': 0.1,
 'Optimizer': 'adam',
 'units1': 36,
 'activation1': 'tanh',
 'dropout1': 0.1,
 'units2': 104,
 'activation2': 'elu',
 'dropout2': 0.9,
 'units3': 56,
 'activation3': 'elu',
 'dropout3': 0.3,
 'units4': 80,
 'activation4': 'elu',
 'dropout4': 0.1,
 'units5': 28,
 'activation5': 'sigmoid',
 'dropout5': 0.5,
 'units6': 108,
 'activation6': 'sigmoid',
 'dropout6': 0.1,
 'units7': 96,
 'activation7': 'relu',
 'dropout7': 0.4,
 'units8': 76,
 'activation8': 'relu',
 'dropout8': 0.2}

In [39]:
model = tuner.get_best_models(num_models =1)[0]

In [40]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 200,
          initial_epoch = 6,
          validation_data = (x_test, y_test))

Epoch 7/200
20/20 [==============================] - 2s 15ms/step - loss: 0.5594 - accuracy: 0.7248 - val_loss: 0.5407 - val_accuracy: 0.7338
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5703 - accuracy: 0.7231 - val_loss: 0.5364 - val_accuracy: 0.7338
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5368 - accuracy: 0.7459 - val_loss: 0.5362 - val_accuracy: 0.7338
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5660 - accuracy: 0.7296 - val_loss: 0.5493 - val_accuracy: 0.7403
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5039 - accuracy: 0.7671 - val_loss: 0.5514 - val_accuracy: 0.7403
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5124 - accuracy: 0.7704 - val_loss: 0.5475 - val_accuracy: 0.7403
Epoch 13/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5252 - accuracy: 0.7459 - val_loss: 0.5393 - val_accuracy: 0.7273
Epoch 14